<a href="https://colab.research.google.com/github/mehdihoore/AstraChatbot/blob/main/telegram_json_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>
.rtl-text {
    direction: rtl;
    text-align: right;
    color: rgb(55, 16, 194);
}
</style>
<div class="rtl-text" dir=rtl>
1-
فایل خود را در سمت چپ آپلود کنید.
</div>
<div class="rtl-text" dir=rtl>
2-
 ماژولهای لازم را نصب کنید.
</div>





In [ ]:
!pip install telethon datetime khayyam google-generativeai pandas
!pip install -U -q "google-generativeai>=0.7.2"
!pip install Pillow PyPDF2 python-docx numpy


<style>
.rtl-text {
    direction: rtl;
    text-align: right;
    color: rgb(55, 16, 194);
}
</style>
<div class="rtl-text" dir=rtl>
3-
فایل خود را از زیپ خارج کنید.
</div>





In [ ]:
!unzip json.zip -d extracted_files/

<style>
.rtl-text {
    direction: rtl;
    text-align: right;
    color: rgb(55, 16, 194);
}
</style>
<div class="rtl-text" dir=rtl>
4-
کد را اجرا کنید.
</div>





In [ ]:
import re
import sys
import csv
import json
import os
from datetime import datetime
from khayyam import JalaliDate
import google.generativeai as genai
import pandas as pd
from typing import Optional, Dict, Any, List
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import io
from PIL import Image
import tempfile
import logging
import requests
from google.colab import userdata

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration for AI processing
class Config:
    def __init__(self):
        # Configure Gemini API key
        # Replace with your actual API key
        self.GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY_NHO")
        genai.configure(api_key=self.GEMINI_API_KEY)

        # Initialize Gemini models
        self.audio_model = genai.GenerativeModel('models/gemini-1.5-flash')
        self.chat_model = genai.GenerativeModel('models/gemini-exp-1121')

# Columns for the CSV output
COLUMNS = [
    "msg_id",
    "sender",
    "sender_id",
    "user_username",
    "user_first_name",
    "user_last_name",
    "user_is_bot",
    "reply_to_msg_id",
    "date",
    "persian_year",
    "persian_month",
    "persian_day",
    "msg_type",
    "msg_content",
    "transcription",
    "doc_summary",
    "img_description",
    "img_features",
    "has_mention",
    "has_email",
    "has_phone",
    "has_hashtag",
    "is_bot_command",
]

persian_months = [
    "فروردین", "اردیبهشت", "خرداد", "تیر", "مرداد", "شهریور",
    "مهر", "آبان", "آذر", "دی", "بهمن", "اسفند"
]

class TelegramJSONProcessor:
    def __init__(self, config: Config):
        self.config = config
        self.processed_messages = []
        self.json_file_path = None  # Store the path to the JSON file

    def process_json_file(self, json_path: str, output_csv_path: str = None):
        """
        Process a JSON file exported from Telegram

        :param json_path: Path to the JSON file containing message history
        :param output_csv_path: Optional custom output CSV path
        """

        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Ensure we are accessing the "messages" key if the JSON is structured
            messages_data = data.get("messages", data)

            # Determine output path
            if not output_csv_path:
                output_csv_path = os.path.join(
                    self.config.OUTPUT_DIR,
                    f"chat_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
                )

            # Process messages
            processed_messages = []
            for message in messages_data:
                if not isinstance(message, dict):
                    logger.warning(f"Skipping non-dictionary item: {message}")
                    continue

                processed_msg = self.process_message(message)
                if processed_msg:
                    processed_messages.append(processed_msg)

            # Save to CSV
            self.save_to_csv(processed_messages, output_csv_path)

            logger.info(f"Processed {len(processed_messages)} messages. CSV saved to {output_csv_path}")

        except Exception as e:
            logger.error(f"Error processing JSON file: {str(e)}")


    def process_message(self, message: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """
        Process individual message with content analysis

        :param message: Message dictionary from JSON
        :return: Processed message dictionary
        """
        json_path = '/content/extracted_files/json/result.json'
        try:
            # Basic message data extraction
            msg_data = {
                "msg_id": message.get('id', 0),
                "sender": message.get('from_name', ''),
                "sender_id": message.get('from_id', ''),
                "user_username": message.get('from_username', None),
                "user_first_name": message.get('from_name', ''),
                "user_last_name": None,
                "user_is_bot": 1 if message.get('from_is_bot', False) else 0,
                "reply_to_msg_id": message.get('reply_to_message_id'),
                "date": message.get('date'),
                "persian_year": 'persian_year',
                "persian_month": 'persian_month',
                "persian_day":'persian_day',
                "msg_type": "text",
                "msg_content": "",
                "transcription": "",
                "doc_summary": "",
                "img_description": "",
                "img_features": "",
                "has_mention": 0,
                "has_email": 0,
                "has_phone": 0,
                "has_hashtag": 0,
                "is_bot_command": 0
            }

            # Handle text content
            if 'text' in message:
                if isinstance(message['text'], list):
                    # Concatenate or handle list of text parts
                    msg_data['msg_content'] = ''.join(
                        part if isinstance(part, str) else '' for part in message['text']
                    )
                elif isinstance(message['text'], str):
                    msg_data['msg_content'] = message['text']
                else:
                    msg_data['msg_content'] = ""

                # Update mentions and hashtags detection
                msg_data['has_mention'] = 1 if '@' in msg_data['msg_content'] else 0
                msg_data['has_hashtag'] = 1 if '#' in msg_data['msg_content'] else 0

            # Process media types
            if 'photo' in message:
                msg_data['msg_type'] = 'image'
                photo_path = message.get('photo', '')

                # Attempt to process image if file exists
                try:
                    full_photo_path = os.path.join(os.path.dirname(json_path),  photo_path)
                    print(full_photo_path)
                    if os.path.exists(full_photo_path):
                        img_analysis = self.process_image(
                            [{'file_path': full_photo_path}])
                        msg_data['img_description'] = img_analysis.get(
                            'description', '')
                        msg_data['img_features'] = ', '.join(
                            img_analysis.get('features', []))
                except Exception as img_err:
                    logger.warning(f"Image processing failed: {str(img_err)}")

            elif 'media_type' in message:
                msg_data['msg_type'] = 'voice_message'
                voice_path = message.get('file', '')
                print("voice path is:", voice_path)

                # Attempt to transcribe voice message
                try:
                    full_voice_path = os.path.join(os.path.dirname(json_path), voice_path)
                    print("voice path is:", full_voice_path)
                    if os.path.exists(full_voice_path):
                        transcription = self.transcribe_voice_message(
                            {'file_path': full_voice_path})
                        msg_data['transcription'] = transcription
                        msg_data['msg_content'] = transcription
                except Exception as voice_err:
                    logger.warning(f"Voice transcription failed: {str(voice_err)}")

            elif 'mime_type' in message:
                msg_data['msg_type'] = 'document'
                doc_path = message.get('file', '')

                # Attempt to process the document
                try:
                    full_doc_path = os.path.join(os.path.dirname(json_path), doc_path)
                    print("full doc path is: ", full_doc_path)
                    if os.path.exists(full_doc_path):
                        doc_summary = self.process_document(
                            {'file_path': full_doc_path})
                        msg_data['doc_summary'] = doc_summary
                except Exception as doc_err:
                    logger.warning(
                        f"Document processing failed: {str(doc_err)}")

            # Regex for email and phone detection
            msg_data['has_email'] = 1 if re.search(
                r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', msg_data['msg_content']) else 0
            msg_data['has_phone'] = 1 if re.search(
                r'\b(?:\+?98|0)?9[0-9]{9}\b', msg_data['msg_content']) else 0
            from dateutil.parser import parse  # Import for flexible date parsing
            # Convert date to Jalali calendar
            if msg_data['date']:
                try:
                    # Parse ISO 8601 date string
                    date_obj = parse(msg_data['date'])
                    jalali_date = JalaliDate(date_obj)
                    msg_data['persian_year'] = jalali_date.year
                    msg_data['persian_month'] = persian_months[jalali_date.month - 1]
                    msg_data['persian_day'] = jalali_date.day
                except Exception as e:
                    logger.warning(f"Date conversion error: {str(e)}")
                    msg_data['persian_year'] = None
                    msg_data['persian_month'] = None
                    msg_data['persian_day'] = None

            return msg_data
            logger.info(f"Processing message: {message.get('id', 'No ID')}")
        except Exception as e:
            logger.error(f"Detailed error processing message: {str(e)}")
            logger.error(f"Problematic message: {message}")
            return None

    def process_image(self, photo_data: List[Dict]) -> Dict[str, Any]:
        """
        Process image using Gemini

        :param photo_data: Photo data from JSON
        :return: Dictionary with image description and features
        """
        print("we are on process_image")
        try:
            # Typically, the last item in the photo list is the largest
            if not photo_data:
                return {"description": "", "features": []}

            # Assume the photo is base64 encoded or has a file_path
            # You might need to adjust this based on your exact JSON structure
            photo = photo_data[-1]
            print("photo is in proccess: ", photo['file_path'])
            # If file path exists, open the image
            if 'file_path' in photo:
                with Image.open(photo['file_path']) as img:
                    # Convert to bytes
                    img_byte_arr = io.BytesIO()
                    img.save(img_byte_arr, format='JPEG')
                    img_byte_arr = img_byte_arr.getvalue()
            else:
                # If base64 encoded, decode
                import base64
                img_byte_arr = base64.b64decode(photo.get('base64', ''))

            # Generate image description
            prompt = """Analyze this image and provide:
            1. A detailed description
            2. Key features or elements present
            Respond in plain text format with two sections:
            DESCRIPTION:
            [Your description here]
            FEATURES:
            - [feature 1]
            - [feature 2]
            etc in persian."""

            # Generate response
            generation_config = {
                "temperature": 1,
                "top_p": 0.95,
                "top_k": 64,
                "max_output_tokens": 8192,

            }
            response = self.config.audio_model.generate_content(
                [prompt, {
                    "mime_type": "image/jpeg",
                    "data": img_byte_arr
                }],
                safety_settings={
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                },
                generation_config=generation_config


            )

            # Parse response
            text = response.text
            description = ""
            features = []

            if "DESCRIPTION:" in text:
                parts = text.split("FEATURES:")
                description = parts[0].replace("DESCRIPTION:", "").strip()
                if len(parts) > 1:
                    features = [f.strip("- ").strip()
                                for f in parts[1].split("\n") if f.strip()]

            return {
                "description": description,
                "features": features
            }

        except Exception as e:
            logger.error(f"Image processing error: {str(e)}")
            return {"description": "Error processing image", "features": []}


    def process_document(self, document_data: Dict) -> str:
        """
        Process document using Gemini

        :param document_data: Document data from JSON
        :return: Document summary
        """
        try:
            # Ensure file path exists
            if not document_data.get('file_path'):
                return "No file path found"

            # Determine file type
            file_ext = os.path.splitext(document_data['file_path'])[1].lower()

            # Read file contents based on type
            text_content = ""
            if file_ext == '.txt':
                with open(document_data['file_path'], 'r', encoding='utf-8') as f:
                    text_content = f.read()
            elif file_ext == '.pdf':
                try:
                    import PyPDF2
                    with open(document_data['file_path'], 'rb') as f:
                        pdf_reader = PyPDF2.PdfReader(f)
                        text_content = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
                except Exception as pdf_err:
                    logger.error(f"PDF extraction error: {pdf_err}")
                    text_content = ""

            # Limit text content
            text_content = text_content.strip()[:2000]

            # Generate summary
            prompt = f"""Please provide a detailed summary of the following document content in persian: {text_content}"""

            response = self.config.chat_model.generate_content(
                prompt,
                safety_settings={
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                }
            )

            return response.text if response else "No summary available"

        except Exception as e:
            logger.error(f"Document processing error: {str(e)}")
            return f"Error processing document: {str(e)}"

    def transcribe_voice_message(self, voice_data: Dict) -> str:
        """
        Transcribe voice message using Gemini

        :param voice_data: Voice message data from JSON
        :return: Transcription text
        """
        try:
            # Ensure file path exists
            if not voice_data.get('file_path'):
                return ""

            # Read audio file
            with open(voice_data['file_path'], 'rb') as audio_file:
                audio_data = audio_file.read()

            # Create prompt
            prompt = "Please transcribe this audio file as plain text in Persian (Farsi) language."

            # Generate transcription
            response = self.config.audio_model.generate_content(
                [
                    prompt,
                    {
                        "mime_type": "audio/ogg",
                        "data": audio_data
                    }
                ],
                safety_settings={
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                },

            )

            return response.text if response and response.text else ""

        except Exception as e:
            logger.error(f"Voice transcription error: {str(e)}")
            return ""

    def save_to_csv(self, messages: List[Dict[str, Any]], output_path: str):
        """
        Save processed messages to CSV

        :param messages: List of processed message dictionaries
        :param output_path: Path to save CSV file
        """
        try:
            # Create DataFrame and save to CSV
            df = pd.DataFrame(messages)
            df.to_csv(output_path, index=False, encoding='utf-8-sig')
        except Exception as e:
            logger.error(f"CSV saving error: {str(e)}")

def main():
    # Initialize configuration
    config = Config()
    GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")
    # Create processor
    processor = TelegramJSONProcessor(config)

    # Check if JSON path is provided as command-line argument

    json_path = '/content/extracted_files/json/result.json'
    output_csv_path = '/content/extracted_files/result.csv'
    processor.process_json_file(json_path, output_csv_path)


if __name__ == "__main__":
    main()





<style>
.rtl-text {
    direction: rtl;
    text-align: right;
    color: rgb(55, 16, 194);
}
</style>
<div class="rtl-text" dir=rtl>
5-
فایل csv تولید را چک کنید
</div>
<div class="rtl-text" dir=rtl>
6-
برای سوال و جواب این کد را ران کنید.
</div>





In [ ]:
import re
import sys
import csv
import json
import os
from datetime import datetime
from khayyam import JalaliDate
import google.generativeai as genai
import pandas as pd
from typing import Optional, Dict, Any, List
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import io
from PIL import Image
import tempfile
import logging
import requests
from google.colab import userdata
from IPython.display import HTML
import sys

class ChatHistoryQA:
    def __init__(self, api_key):
        """
        Initialize the ChatHistoryQA with Gemini API configuration

        :param api_key: Google Gemini API key
        """
        genai.configure(api_key=api_key)
        self.chat_model = genai.GenerativeModel('models/gemini-exp-1121')

    def create_comprehensive_context(self, df, max_length=10000):
        """
        Create a comprehensive context from the DataFrame

        :param df: DataFrame containing chat history
        :param max_length: Maximum length of context
        :return: Formatted context string
        """
        context_parts = []

        for _, row in df.iterrows():
            # Create context entry with multiple details
            context_entry = [
                f"پیام {row['msg_id']}",
                f"فرستنده: {self._get_user_reference(row)}",
                f"تاریخ شمسی: {int(row['persian_year'])} {row['persian_month']} {int(row['persian_day'])}",
                f"تاریخ میلادی: {row['date']}"
            ]

            # Add content from different message types
            if pd.notna(row['msg_content']) and str(row['msg_content']).strip():
                context_entry.append(f"متن پیام: {row['msg_content']}")

            if pd.notna(row['transcription']) and str(row['transcription']).strip():
                context_entry.append(f"رونویسی صوتی: {row['transcription']}")

            if pd.notna(row['doc_summary']) and str(row['doc_summary']).strip():
                context_entry.append(f"خلاصه سند: {row['doc_summary']}")

            if pd.notna(row['img_description']) and str(row['img_description']).strip():
                context_entry.append(f"توصیف تصویر: {row['img_description']}")

            context_parts.append(" | ".join(context_entry))

            # Break if context gets too long
            if len(" ".join(context_parts)) > max_length:
                break

        return "\n".join(context_parts)

    def _get_user_reference(self, row):
        """
        Generate a user reference with privacy protection

        :param row: DataFrame row
        :return: User reference string
        """
        # Priority 1: Username
        if pd.notna(row['user_username']) and str(row['user_username']).strip():
            return f"@{row['user_username']}"

        # Priority 2: Name
        name_parts = []
        if pd.notna(row['user_first_name']) and str(row['user_first_name']).strip() not in ['.', '']:
            name_parts.append(row['user_first_name'])
        if pd.notna(row['user_last_name']) and str(row['user_last_name']).strip() not in ['.', '']:
            name_parts.append(row['user_last_name'])

        if name_parts:
            return " ".join(name_parts)

        # Priority 3: Sender ID
        if pd.notna(row['sender_id']):
            return f"کاربر {row['sender_id']}"

        # Default
        return "کاربر ناشناس"

    def ask_question(self, csv_path: str, question: str):
        """
        Ask a question about the chat history

        :param csv_path: Path to the CSV file
        :param question: Question to ask
        :return: AI-generated response
        """
        try:
            # Read CSV file
            df = pd.read_csv(csv_path, encoding='utf-8-sig')

            # Prepare comprehensive context
            context = self.create_comprehensive_context(df)

            # Construct prompt
            prompt = f"""System Instructions:
            You are an AI assistant analyzing group discussions. Provide detailed analysis based on the available context.

            Context of Conversations:
            {context}

            Question: {question}

            Please provide a detailed answer based on the conversation history, citing specific messages or details when possible."""

            # Generate response
            response = self.chat_model.generate_content(
                prompt,
                generation_config={
                    'temperature': 0.3,
                    'top_p': 0.8,
                },
                safety_settings={
                    genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
                    genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                }
            )

            return f"\u200F{response.text}\u200F"

        except Exception as e:
            return f"\u200Fخطا در پردازش سوال: {str(e)}\u200F"

# Usage example
def main():
    # Replace with your actual Google API key
    API_KEY = userdata.get("GOOGLE_API_KEY_BHR1")

    # Replace with the path to your CSV file
    CSV_PATH = "/content/extracted_files/result.csv"

    # Create QA instance
    qa = ChatHistoryQA(API_KEY)

    # Interactive Q&A loop
    while True:
        question_html = HTML(f"""
        <div dir='rtl' style='text-align: right;'>
            سوال خود را بپرسید (برای خروج 'exit' را وارد کنید):
        </div>
        """)
        display(question_html)
        question = input()

        if question.lower() == 'exit':
            break

        # Ask question and display response
        try:
            response = qa.ask_question(CSV_PATH, question)
            response_html = HTML(f"""
            <div dir='rtl' style='text-align: right;'>
                <h3>پاسخ:</h3>

            </div>
            <p>حالا در ادامه بشنوید از پاسخ:</p>
            <div>{response}</div>
            """)
            display(response_html)

        except Exception as e:
            error_html = HTML(f"""
            <div dir='rtl' style='text-align: right; color: red;'>
                <h3>خطا:</h3>
                <p>{str(e)}</p>
            </div>
            """)
            display(error_html)

        separator_html = HTML("<hr/>")
        display(separator_html)
if __name__ == "__main__":
    main()